# Machine Learning en Julia

El objetivo de ésta semana es el de tratar el area del machine learning en Julia, ver sus metodos, ventajas y posibles dificultades,se evaluará en particular el rendimiento de los modelos y buscamos construir un panorama general.

In [1]:
#Importando librerías
include("extras.jl") # Script propio, importado externamente
using CSV,DataFrames,Random,Statistics,BenchmarkTools
using .extras

### Importando datos y preparandolos.

In [2]:
path=pwd()*"/Data/clean_dataset.csv" 
df = CSV.File(path)|> DataFrame
first(df,8)

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity
,Int64,Float64,Float64,Int64,Int64,String31,String7
1,1,30.83,0.0,1,1,Industrials,White
2,0,58.67,4.46,1,1,Materials,Black
3,0,24.5,0.5,1,1,Materials,Black
4,1,27.83,1.54,1,1,Industrials,White
5,1,20.17,5.625,1,1,Industrials,White
6,1,32.08,4.0,1,1,CommunicationServices,White
7,1,33.17,1.04,1,1,Transport,Black
8,0,22.92,11.585,1,1,InformationTechnology,White


In [3]:
#Seleccionamos lo datos
select!(df,extras.detect_strings(df)) #Para usar el modulo que importamos usamos la sintaxis NombreDelModulo.funcion)()

,Gender,Age,Debt,Married,BankCustomer,YearsEmployed,PriorDefault,Employed
,Int64,Float64,Float64,Int64,Int64,Float64,Int64,Int64
1,1,30.83,0.0,1,1,1.25,1,1
2,0,58.67,4.46,1,1,3.04,1,1
3,0,24.5,0.5,1,1,1.5,1,0
4,1,27.83,1.54,1,1,3.75,1,1
5,1,20.17,5.625,1,1,1.71,1,0
6,1,32.08,4.0,1,1,2.5,1,0
7,1,33.17,1.04,1,1,6.5,1,0
8,0,22.92,11.585,1,1,0.04,1,0
9,1,54.42,0.5,0,0,3.96,1,0


In [4]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Gender,0.695652,0,1.0,1,0,Int64
2,Age,31.5141,13.75,28.46,80.25,0,Float64
3,Debt,4.75872,0.0,2.75,28.0,0,Float64
4,Married,0.76087,0,1.0,1,0,Int64
5,BankCustomer,0.763768,0,1.0,1,0,Int64
6,YearsEmployed,2.22341,0.0,1.0,28.5,0,Float64
7,PriorDefault,0.523188,0,1.0,1,0,Int64
8,Employed,0.427536,0,0.0,1,0,Int64
9,CreditScore,2.4,0,0.0,67,0,Int64


Los dividimos en entrenamiento y test

In [5]:
sample = randsubseq(1:size(df,1), 0.75)
train = df[sample, :]
notsample = [i for i in 1:size(df,1) if isempty(searchsorted(sample, i))]
test = df[notsample, :]
first(train,3)

,Gender,Age,Debt,Married,BankCustomer,YearsEmployed,PriorDefault,Employed
,Int64,Float64,Float64,Int64,Int64,Float64,Int64,Int64
1,1,30.83,0.0,1,1,1.25,1,1
2,0,58.67,4.46,1,1,3.04,1,1
3,0,24.5,0.5,1,1,1.5,1,0


In [6]:
X_train = Array(train[:, 1:12])
y_train = Array(train[:, 13])

X_test = Array(test[:, 1:12])
y_test = Array(test[:, 13])

166-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

## Regresión logistica

In [7]:
using GLM

Aquí usamos la función formula que nos dice que
$$A\approx f(\alpha_{1}G +\alpha_{2}A \alpha_{3}D +\cdot +\alpha_{12}I )$$

In [8]:
fm = @formula(Approved ~ Gender+ Age+ Debt+ Married+ BankCustomer+ YearsEmployed+ PriorDefault+ Employed+ CreditScore+ DriversLicense+ ZipCode+ Income)

FormulaTerm
Response:
  Approved(unknown)
Predictors:
  Gender(unknown)
  Age(unknown)
  Debt(unknown)
  Married(unknown)
  BankCustomer(unknown)
  YearsEmployed(unknown)
  PriorDefault(unknown)
  Employed(unknown)
  CreditScore(unknown)
  DriversLicense(unknown)
  ZipCode(unknown)
  Income(unknown)

Usamos benchmark para ver el desempeño del algoritmo a nivel computacional con los valores por defecto, en éste caso es una regresión logistica con el paquete **GLM**

In [9]:
@benchmark glm(fm, train , Binomial(),ProbitLink())

BenchmarkTools.Trial: 1247 samples with 1 evaluation.
 Range (min … max):  1.820 ms … 34.184 ms  ┊ GC (min … max): 0.00% … 17.28%
 Time  (median):     2.045 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.983 ms ±  4.295 ms  ┊ GC (mean ± σ):  1.61% ±  5.48%

  █▅▁▁▁                                                       
  █████████▇▅▅█▇██▇██▇▆▇▇▆▃▇▆▆▅▅▅▆▃▅▄▆▆▁▅▄▄▆▄▁▁▅▁▁▃▄▃▄▃▁▁▁▁▄ ▇
  1.82 ms      Histogram: log(frequency) by time     23.1 ms <

 Memory estimate: 458.22 KiB, allocs estimate: 1375.

Entrenamos el modelo con un modelo lineal generalizado

In [10]:
logit= glm(fm, train , Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Approved ~ 1 + Gender + Age + Debt + Married + BankCustomer + YearsEmployed + PriorDefault + Employed + CreditScore + DriversLicense + ZipCode + Income

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                       Coef.    Std. Error      z  Pr(>|z|)     Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)     -1.46375       0.296658     -4.93    <1e-06   -2.04519     -0.882307
Gender          -0.081235      0.165125     -0.49    0.6227   -0.404874     0.242404
Age             -0.00560027    0.00661872   -0.85    0.3975   -0.0185727    0.00737217
Debt            -0.0347453     0.0162674    -2.14    0.0327   -0.0666289   -0.00286164
Married    

In [11]:
pred=predict(logit,test)
prediction_class=[if x<0.5 0 else 1 end for x in pred]
df_result=test[:,"Approved"]
println("Terminado")

Terminado


In [12]:
prediction_correctly= prediction_class.== df_result
accuracy=mean(prediction_correctly)*100
println("El modelo tuvo una precisión del "*string(accuracy)*"%" )

El modelo tuvo una precisión del 89.1566265060241%


## Arboles de decisión

In [13]:
using DecisionTree

In [14]:
model = DecisionTreeClassifier(max_depth=5)

DecisionTreeClassifier
max_depth:                5
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [15]:
using DecisionTree:fit!

In [ ]:
@benchmark fit!(model, X_train, y_train)

In [ ]:
fit!(model, X_train, y_train)

In [ ]:
# Predict
dectree_pred = DecisionTree.predict(model, X_test)

extras.evaluate(y_test,dectree_pred)

# Compute accuracy
correct = 0
n=length(y_test)
for i in 1:n
    if y_test[i] == dectree_pred[i]
        correct = correct + 1
    end
end
println(correct / n)

## Random Forest

In [ ]:
# Fit the model
rf = RandomForestClassifier()
fit!(rf, X_train, y_train)
@benchmark fit!(rf, X_train, y_train)


In [ ]:

# Predict on the test set
rf_pred = DecisionTree.predict(rf, X_test)

# Compute accuracy
correct = 0
n=length(y_test)
for i in 1:n
    if y_test[i] == rf_pred[i]
        correct = correct + 1
    end
end
println(correct / n)

## ScikitLearn!

In [ ]:
# Import the model you want to use
using ScikitLearn
@sk_import linear_model: LogisticRegression
using ScikitLearn:fit!

In [ ]:
# Fit the model
@benchmark  fit!(LogisticRegression(), X_train, y_train)

In [ ]:
log_reg = fit!(LogisticRegression(), X_train, y_train)

In [ ]:


# Predict on the test set
sklearn_pred = log_reg.predict(X_test)

# Compute the accuracy
correct = 0
n=length(y_test)
for i in 1:n
    if y_test[i] == sklearn_pred[i]       
        correct = correct + 1
    end
end
println(correct / n)

## MLJ

In [ ]:
using MLJ

In [ ]:
svc_model = (@load SVC verbosity=1)()


# create a so-called machine
svc = machine(svc_model, X_train, categorical(y_train))
using MLJ:fit!
# fit the model
MLJ.fit!(svc);

In [ ]:
models()

In [ ]:
ϕ=MLJ.predict(svc, X_test);

In [ ]:
#compute the accuracy
correct = 0
n=length(y_test)
for i in 1:n
    if y_test[i] == ϕ[i]       
        correct = correct + 1
    end
end
println(correct / n)

In [ ]:
models("support")

In [ ]:
load_path("SVMClassifier", pkg="ScikitLearn")

In [ ]:
svm_model = (@load SVMClassifier pkg=ScikitLearn verbosity=1)()


# create a so-called machine
svm = machine(svm_model, X_train, categorical(y_train))
using MLJ:fit!
# fit the model
MLJ.fit!(svm);

In [ ]:
ϕ2=MLJ.predict(svm, X_test);
#compute the accuracy
correct = 0
n=length(y_test)
for i in 1:n
    if y_test[i] == ϕ2[i]       
        correct = correct + 1
    end
end
println(correct / n)

```Julia
Pkg.add("BenchmarkTools")

Pkg.add("DecisionTree")
Pkg.add("ScikitLearn")

Pkg.add("MLJ")
Pkg.add("LIBSVM")
Pkg.add("MLJModels")

Pkg.add("MLJScikitLearnInterface")
```